In [1]:
%config IPCompleter.greedy=True

In [2]:
# %load_ext snakeviz

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# native modules
import sys
import datetime
from os import path, listdir
from random import choice, sample
import re
import itertools
from importlib import reload

In [18]:
# data study
import pandas as pd
import numpy as np
from copy import deepcopy
import requests
import json

In [6]:
# multithreading
# import concurrent.futures
# import threading

In [7]:
# charting
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Recon project
# sys.path.append(r'C:\Users\kklekota\PycharmProjects\Recon')
# import utils.xml_parser as parser

In [71]:
# mkt data project
# sys.path.append(r'C:\Users\kklekota\PycharmProjects\mkt_data')
from mkt_data.mkt_data import const, helper
import mkt_data.mkt_data as md

In [ ]:
# macs service
# from macs_service import Client, generate_json_data
# client = Client("http://leviathan33:5000")

In [70]:
%%HTML
<iframe src="http://ci-macsi/jenkins/view/MACS%20services%20status/" 
 width="600"  height="450" frameborder="0" scrolling="no"></iframe>

In [ ]:
# service project
# sys.path.append(r'C:\Users\kklekota\PycharmProjects\Service')
# import foxutils as fu
# import h5helper
# import mxgrid_replay as replay

In [122]:
from fox_toolbox.sprd import sprd
from fox_toolbox.utils import xml_parser
from fox_toolbox.utils.rates import Swaption
from fox_toolbox.sprd import tsr

In [11]:
help(sprd.get_premium)

Help on function get_premium in module fox_toolbox.sprd.sprd:

get_premium(fwd1:float, vol1:fox_toolbox.utils.rates.Volatility, fwd2:float, vol2:fox_toolbox.utils.rates.Volatility, corr:float, expiry:float, strike:float)



In [143]:
fpath = r"D:\Work\BGM_TESTS\FMT-66\results\20190408_mc_4770216_smile_sabr_2f_B_SKEW_CURVE_FREEZING\irsms\JC202_00_RatesBump.xml"
_, irsmform = xml_parser.get_xml(fpath)
dsc_curve, sprd_curves = xml_parser.get_curves(irsmform)
fwd_curve = sprd_curves[0]

In [144]:
fpath = r"D:\Work\BGM_TESTS\FMT-66\results\20190408_mc_4770216_smile_sabr_2f_B_SKEW_CURVE_FREEZING\irsms\JC202_00_RatesBump.result.xml"
_, irsmout = xml_parser.get_xml(fpath)

In [145]:
cal_bskt = [x for x in xml_parser.get_calib_basket(irsmout) if x.cal_premium > 0.0]

In [146]:
len(cal_bskt)

28

In [147]:
cal_bskt[0].__dict__

{'_dcfs': array([0.50136986, 0.49863014, 0.50136986, 0.50136986]),
 '_expiry': 0.4986301369863,
 '_libor_tenor': 6,
 '_pmnt_dates': array([1.0109589 , 1.50958904, 2.0109589 , 2.51232877]),
 '_start_date': 0.5095890410959,
 '_vol': Volatility(value=0.0749, type='SLN', shift_size=0.01),
 'annuity': 2.0037620355137,
 'cal_premium': 0.0004480662123,
 'cal_type': 'ATM',
 'cal_vol': 0.0750255964136,
 'fwd': 0.0005812762924,
 'fwdAdj': 0.0005821626561,
 'fwdAdjModel': 0.0005817683548,
 'lvl': 0,
 'strike': 0.0005812762923760001,
 'tgt_premium': nan}

In [148]:
correl_node = irsmout.find('.//BGMCMSSOCalib')
# sprd_nodes = list(xml_parser.findall('.//CMSSpreadCaplet'))
correls = xml_parser.get_delim_float_node(correl_node, './/CMSSOMarketCorr//Values')
premiums = xml_parser.get_delim_float_node(correl_node, './/CMSSOMarketPrice//Values')

In [149]:
mr = xml_parser.get_float_node(irsmform, './/MeanRR')
print(f'Mean reversion: {mr}')

Mean reversion: 0.03


In [150]:
def print_repl_quality(label, ref_label, ref_value, repl_label, repl_value, is_nominal_adj=True):
    mult = 1e4 if is_nominal_adj else 1.0
    REPL_FMT_STR = '{}:\n\t{:<15}{}\n\t{:<15}{}\n\t{:<15}{}'
    print(REPL_FMT_STR.format(label, ref_label, mult * ref_value, 
                              repl_label, mult * repl_value, 
                              "difference", mult * (ref_value - repl_value)))

In [142]:
dic_ = {}
eps = 1e-4

size = int(len(cal_bskt) / 2)
for i in range(size):
    instr2 = cal_bskt[i]
    instr = cal_bskt[i + size]
    strike = instr1.fwd - instr2.fwd
    
    adj_tsr_short = instr2.fwdAdj - instr2.fwd
    adj_bgm_short = instr2.fwdAdjModel - instr2.fwd
        
    fwd = instr.fwd
    adj_tsr = instr.fwdAdj - fwd
    adj_bgm = instr.fwdAdjModel - fwd
    
    a, _ = tsr.get_linear_coeffs(instr, dsc_curve, mr, instr.start_date, fwd_curve)
    coeff = instr.get_annuity(dsc_curve) / dsc_curve.get_dsc(instr.start_date)
    variance = adj_tsr / (coeff * a)
    tol = eps #* math.sqrt(instr.expiry)
    
    tgt_adj = instr.fwdAdjModel - instr2.fwdAdjModel + instr2.fwdAdj - fwd
    if tgt_adj < adj_tsr:
        tgt_adj = min(adj_tsr, tgt_adj + tol)
    else:
        tgt_adj = max(adj_tsr, tgt_adj - tol)
    
    tgt_a = tgt_adj / (coeff * variance)
    impl_mr = tsr.get_impl_mr(tgt_a, instr, dsc_curve, instr.start_date, fwd_curve)
    
    print(f'{instr.expiry:0.2f}\t{adj_tsr:0.6f}\t{adj_bgm:0.6f}\t{mr:0.2f}\t{impl_mr:0.6f}')
    
    dic_.setdefault('Expiry', []).append(instr2.expiry)
    dic_.setdefault('MR', []).append(impl_mr)
    
df = pd.DataFrame(dic_)
df['ExpiryLbl'] = df['Expiry'].apply(float_to_lbl)
df.to_clipboard()

0.50	0.000047	0.000024	0.03	0.030000
1.01	0.000109	0.000059	0.03	0.030000
1.50	0.000192	0.000100	0.03	0.030000
2.01	0.000270	0.000146	0.03	0.002996
3.00	0.000466	0.000255	0.03	-0.046424
4.00	0.000685	0.000382	0.03	-0.064638
5.00	0.000939	0.000527	0.03	-0.074773
6.00	0.001258	0.000696	0.03	-0.083691
7.01	0.001679	0.000882	0.03	-0.096258
8.01	0.002120	0.001081	0.03	-0.102544
9.01	0.002582	0.001291	0.03	-0.107114
10.01	0.003045	0.001510	0.03	-0.109602
15.01	0.008138	0.002621	0.03	-0.196298
20.01	0.015694	0.003715	0.03	-0.183998


In [101]:
size = int(len(cal_bskt) / 2)
for i in range(size):
    instr2 = cal_bskt[i]
    instr1 = cal_bskt[i + size]
    strike = instr1.fwd - instr2.fwd
    dsc = dsc_curve.get_dsc(instr2.expiry)
    
    
    
    premium_tsr = dsc * sprd.get_premium(instr1.fwdAdj, instr1.vol, instr2.fwdAdj, instr2.vol, correls[i], instr2.expiry, strike) * 1e4
    premium_tgt = premiums[i] * 1e4
    err = premium_tsr - premium_tgt
    print(f'{instr2.expiry:0.2f}\t{premium_tgt:0.2f}\t{premium_tsr:0.2f}\t{err:0.6f}')

0.50	5.35	5.35	0.000352
1.01	8.23	8.23	-0.000372
1.50	10.40	10.40	-0.000296
2.01	12.28	12.28	-0.000220
3.00	15.45	15.45	-0.000026
4.00	18.14	18.14	-0.000059
5.00	20.92	20.92	0.000009
6.00	23.87	23.87	0.000482
7.01	27.22	27.22	0.000220
8.01	30.42	30.42	0.000540
9.01	33.67	33.67	0.000406
10.01	36.79	36.79	-0.000010
15.01	70.12	70.12	0.000043
20.01	106.66	106.66	-0.000126


In [151]:
dic_ = {}

size = int(len(cal_bskt) / 2)
for i in range(size):
    instr2 = cal_bskt[i]
    instr1 = cal_bskt[i + size]
    strike = instr1.fwd - instr2.fwd
    dsc = dsc_curve.get_dsc(instr2.expiry)
    
    corr_tsr = correls[i]
    premium_tsr = dsc * sprd.get_premium(instr1.fwdAdj, instr1.vol, instr2.fwdAdj, instr2.vol, corr_tsr, instr2.expiry, strike) * 1e4
    premium_bgm = dsc * sprd.get_premium(instr1.fwdAdjModel, instr1.vol, instr2.fwdAdjModel, instr2.vol, corr_tsr, instr2.expiry, strike) * 1e4
    nd_tgt_premium = premium_tsr / dsc / 1e4
    impl_corr_bgm = sprd.get_impl_corr(nd_tgt_premium, instr1.fwdAdjModel, instr1.vol, 
                                       instr2.fwdAdjModel, instr2.vol, instr2.expiry, strike)
    
    print(f'{instr2.expiry:0.2f}\t{premium_tsr:0.2f}\t{premium_bgm:0.2f}\t{premium_tsr-premium_bgm:0.6f}\t{corr_tsr}\t{impl_corr_bgm}')
    
    
    dic_.setdefault('Expiry', []).append(instr2.expiry)
    
    dic_.setdefault('RateLong', []).append(instr1.fwd)
    dic_.setdefault('RateShort', []).append(instr2.fwd)
    
    dic_.setdefault('TSRRateLong', []).append(instr1.fwdAdj)
    dic_.setdefault('BGMRateLong', []).append(instr1.fwdAdjModel)
    
    dic_.setdefault('TSRRateShort', []).append(instr2.fwdAdj)
    dic_.setdefault('BGMRateShort', []).append(instr2.fwdAdjModel)
    
    dic_.setdefault('TSRPremium', []).append(premium_tsr)
    dic_.setdefault('BGMPremium', []).append(premium_bgm)
    
    dic_.setdefault('TSRCorr', []).append(corr_tsr)
    dic_.setdefault('BGMImpliedCorr', []).append(impl_corr_bgm)

0.50	5.37	5.23	0.145367	0.7617049381198	0.7106218905086832
1.01	8.23	7.96	0.266628	0.7729554050119	0.7267717125731636
1.50	10.33	9.90	0.431146	0.7835401898504	0.733325251805418
2.01	12.14	11.60	0.536925	0.7936070795779	0.7472117789875863
3.00	15.03	14.28	0.747931	0.8122499820006	0.7717150056256233
4.00	17.38	16.45	0.928802	0.8291651326575	0.7940169449848637
5.00	19.68	18.62	1.061836	0.8444705939037	0.8135608390433596
6.00	21.95	20.74	1.201922	0.8583195479395	0.830271595479515
7.01	24.01	22.83	1.177705	0.8709438177369	0.8483314143955547
8.01	26.28	24.71	1.569641	0.8822453957329	0.8565583331250137
9.01	28.08	26.54	1.542835	0.8924741619497	0.8707602825521472
10.01	30.08	28.22	1.856651	0.9017470051627	0.8788222098284555
15.01	38.09	36.32	1.768845	0.936494944075	0.9236099594639958
20.01	47.52	42.36	5.155598	0.957554545909	0.9291045655058343


In [152]:
df = pd.DataFrame(dic_)
df['ExpiryLbl'] = df['Expiry'].apply(float_to_lbl)
df.to_clipboard()

In [106]:
import math
def float_to_lbl(x):
    tmp = (round(x * 12.)) / 12.
    years = int(math.floor(tmp))
    months = int(round((tmp - years) * 12.))
    res = ''
    if years > 0:
        res += '{}Y'.format(years)
    if months > 0:
        res += '{}M'.format(months)
    return res

In [73]:
const.CMS_DIC

{'CMS_AS': 'CMS spread ATM Straddle quotes',
 'CMS_CF': 'CMS spread Cap/Floor quotes',
 'CMS_SLAS': 'CMS spread single-look ATM Straddle',
 'CMS_SW': 'CMS swap'}

In [74]:
df = md.get_store()['CMS_SLAS']

In [75]:
df.head()

RICNAME SPREAD EXPIRY  CCY  SP_PRM  STRIKE  SL CORR  \
DATE       CCY                                                               
2016-08-29 EUR  CO1SOEUR11001Y    110    01Y  EUR    36.2    76.2    0.602   
           EUR  CO1SOEUR11002Y    110    02Y  EUR    48.5    90.4    0.753   
           EUR  CO1SOEUR11003M    110    03M  EUR    18.6    63.3    0.306   
           EUR  CO1SOEUR11003Y    110    03Y  EUR    55.9   102.1    0.811   
           EUR  CO1SOEUR11005Y    110    05Y  EUR    67.0    97.6    0.857   

                SL NVOL        DATE  
DATE       CCY                       
2016-08-29 EUR    45.32  2016-08-29  
           EUR    42.85  2016-08-29  
           EUR    46.29  2016-08-29  
           EUR    40.19  2016-08-29  
           EUR    37.27  2016-08-29

In [77]:
tmp = df.loc[df.DATE.max(), 'USD']
tmp.head()

C:\Users\kklekota\AppData\Local\Continuum\anaconda3\envs\p3test\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


RICNAME SPREAD EXPIRY  CCY  SP_PRM  STRIKE  SL CORR  \
DATE       CCY                                                               
2019-03-29 USD  CO1SOUSD11001Y    110    01Y  USD    28.8    21.7    0.814   
           USD  CO1SOUSD11002Y    110    02Y  USD    46.2    38.3    0.778   
           USD  CO1SOUSD11003M    110    03M  USD    13.4    -5.0    0.818   
           USD  CO1SOUSD11003Y    110    03Y  USD    54.2    39.2    0.803   
           USD  CO1SOUSD11005Y    110    05Y  USD    64.1    36.7    0.832   

                SL NVOL        DATE  
DATE       CCY                       
2019-03-29 USD    36.71  2019-03-29  
           USD    42.65  2019-03-29  
           USD    33.21  2019-03-29  
           USD    41.61  2019-03-29  
           USD    39.71  2019-03-29

In [80]:
tmp.pivot_table(index='SPREAD', columns='EXPIRY', values='SL CORR', aggfunc='count')

EXPIRY,01Y,02Y,03M,03Y,05Y,06M,07Y,10Y,15Y,18M,20Y
SPREAD,,,,,,,,,,,
110,1,1,1,1,1,1,1,1,1,1,1
120,1,1,1,1,1,1,1,1,1,1,1
130,1,1,1,1,1,1,1,1,1,1,1
205,1,1,1,1,1,1,1,1,1,1,1
210,1,1,1,1,1,1,1,1,1,1,1
220,1,1,1,1,1,1,1,1,1,1,1
230,1,1,1,1,1,1,1,1,1,1,1
510,1,1,1,1,1,1,1,1,1,1,1
530,1,1,1,1,1,1,1,1,1,1,1


In [86]:
tmp = df.loc[df.DATE.max(), 'EUR']
tmp = tmp[tmp.SPREAD == '220']
tmp['EXP_FLT'] = tmp['EXPIRY'].apply(lbl_to_float)
tmp.to_clipboard()

C:\Users\kklekota\AppData\Local\Continuum\anaconda3\envs\p3test\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


In [83]:
def lbl_to_float(lbl):
    res = float(lbl[:-1])
    suffix = lbl[-1]
    if suffix == 'Y': 
        return res
    if suffix == 'M':
        return res / 12.0
    raise NotImplemented()

In [85]:
df.CCY.unique()

array(['EUR', 'USD', nan], dtype=object)

In [92]:
tmp = df.loc[df.DATE.max(), 'EUR']
tmp['EXP_FLT'] = tmp['EXPIRY'].apply(lbl_to_float)

C:\Users\kklekota\AppData\Local\Continuum\anaconda3\envs\p3test\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.
C:\Users\kklekota\AppData\Local\Continuum\anaconda3\envs\p3test\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [93]:
tmp.pivot_table(index='EXP_FLT', columns='SPREAD', values='SP_PRM').to_clipboard()

In [91]:
df.DATE.max()

datetime.date(2019, 3, 29)

In [108]:
df = md.get_store()['CMS_AS']

In [109]:
tmp = df.loc[df.DATE.max()]

In [110]:
tmp.head()

,ATM_CORRELATION,ATM_CORRELEATION,"ATM_CORRELEATION""",ATM_STRIKE,CCY,CLOSING_STRIKE,DATE,EXPIRY,RIC,SPREAD,STRADDLE
CCY,,,,,,,,,,,
EUR,0.712,NaN,NaN,38.0,EUR,NaN,2019-03-29,01Y,COSOAEUR10501Y,105,9.8
EUR,0.898,NaN,NaN,46.3,EUR,NaN,2019-03-29,05Y,COSOAEUR10505Y,105,115
EUR,0.930,NaN,NaN,41.8,EUR,NaN,2019-03-29,10Y,COSOAEUR10510Y,105,316.3
EUR,0.933,NaN,NaN,25.4,EUR,NaN,2019-03-29,20Y,COSOAEUR10520Y,105,935.5
EUR,0.543,NaN,NaN,85.1,EUR,NaN,2019-03-29,01Y,COSOAEUR11001Y,110,13.9


In [112]:
tmp.columns

Index(['ATM_CORRELATION', 'ATM_CORRELEATION', 'ATM_CORRELEATION"',
       'ATM_STRIKE', 'CCY', 'CLOSING_STRIKE', 'DATE', 'EXPIRY', 'RIC',
       'SPREAD', 'STRADDLE'],
      dtype='object')

In [119]:
fpath = r"D:\Work\Issues\macs\20190408_tsr_debug\irsmform.xml"
_, irsmform = xml_parser.get_xml(fpath)
dsc_curve, sprd_curves = xml_parser.get_curves(irsmform)

In [114]:
fpath = r"D:\Work\Issues\macs\20190408_tsr_debug\irsmout.xml"
_, irsmout = xml_parser.get_xml(fpath)

In [115]:
cal_bskt = [x for x in xml_parser.get_calib_basket(irsmout) if x.cal_premium > 0.0]

In [116]:
cal_bskt[0].__dict__

{'_dcfs': array([1.01388889, 1.01388889, 1.01944444, 1.01111111, 1.01388889,
        1.01388889, 1.01666667, 1.01944444, 1.01111111, 1.01111111]),
 '_expiry': 5.0027397260274,
 '_libor_tenor': 6,
 '_pmnt_dates': array([ 6.00821918,  7.00821918,  8.01369863,  9.0109589 , 10.0109589 ,
        11.0109589 , 12.01369863, 13.01917808, 14.01643836, 15.01369863]),
 '_start_date': 5.0082191780822,
 '_vol': Volatility(value=0.195729, type='SLN', shift_size=0.02),
 'annuity': 9.2110240048811,
 'cal_premium': 0.0067790592705,
 'cal_type': 'ATM',
 'cal_vol': 0.195729,
 'fwd': 0.0194777798149,
 'fwdAdj': 0.0223124122594,
 'fwdAdjModel': 0.0,
 'lvl': 0,
 'strike': 0.019474121174592,
 'tgt_premium': nan}

In [118]:
from fox_toolbox.sprd import tsr

In [121]:
mr = 0.0400000000000
pmnt = 6.008219
tsr_params = tsr.get_linear_coeffs(cal_bskt[0], dsc_curve, mr, pmnt, sprd_curves[0])
tsr_params

(0.4787268355748829, 0.09717395286506701)

In [120]:
help(tsr.get_linear_coeffs)

Help on function get_linear_coeffs in module fox_toolbox.sprd.tsr:

get_linear_coeffs(cal_instr, main_curve, mr:float, pmnt_date:float, fwd_curve=None)



In [ ]:
a ;   0.4787268025896 
b ;   0.0971739573781 